<a href="https://colab.research.google.com/github/sfwtopoulos/stocks/blob/master/Stocks_seq_1st_argo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import math
import warnings
import numpy as np
import time
import sys
import talos
import pandas_datareader as pdr
#%tensorflow_version 2.x
# Restart runtime using 'Runtime' -> 'Restart runtime...'
%tensorflow_version 1.x
import tensorflow as tf
import keras


# from keras.layers import LSTM
# from keras.models import Sequential
# from keras.layers.wrappers import TimeDistributed
# from keras.layers.wrappers import TimeDistributed
# from keras.layers.core import Dense, Activation, Dropout
# from keras.preprocessing.sequence import pad_sequences

from tensorflow.python.keras.layers import Dense, Activation, Dropout, Input
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.wrappers import TimeDistributed
from tensorflow.python.keras.layers.wrappers import TimeDistributed
from tensorflow.python.keras.preprocessing.sequence import pad_sequences


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [0]:
#pip install talos

In [0]:
#!pip install talos -q

In [0]:
# from keras.optimizers import Adam, Nadam
# from keras.activations import softmax
# from keras.losses import categorical_crossentropy, logcosh
# from talos.utils import lr_normalizer

from tensorflow.python.keras.optimizers import Adam, Nadam
from tensorflow.python.keras.activations import softmax
from tensorflow.python.keras.losses import categorical_crossentropy, logcosh
from talos.utils import lr_normalizer

In [69]:
print(keras.__version__)
print(tf.__version__)
#print(tensorflow.compat.v2.__version__)

2.3.0
1.14.0


In [0]:
#Data import from git repo
url = 'https://raw.githubusercontent.com/sfwtopoulos/stocks/master/stocks_dataset/combined.csv'
#df1 = pd.read_csv(url, error_bad_lines=False)
dfstocks = pd.read_csv(url, sep=',')

In [71]:
#fix column names
dfstocks=dfstocks.rename({' AAPL':'Company', ' Close/Last':'Close', ' Volume':'Volume', ' Open':'Open', ' High':'High', ' Low':'Low'}, axis=1);
for col in dfstocks.columns: 
    print(col) 

Date
Close
Volume
Open
High
Low
Company


In [0]:
#drop $sign from values
dfstocks.Close=dfstocks['Close'].astype(str)
dfstocks.Close=dfstocks.Close.apply(lambda x: x.replace('$',''))
dfstocks.Open=dfstocks['Open'].astype(str)
dfstocks.Open=dfstocks.Open.apply(lambda x: x.replace('$',''))
dfstocks.High=dfstocks['High'].astype(str)
dfstocks.High=dfstocks.High.apply(lambda x: x.replace('$',''))
dfstocks.Low=dfstocks['Low'].astype(str)
dfstocks.Low=dfstocks.Low.apply(lambda x: x.replace('$',''))

In [73]:
#drop rows containing nan or header from the csv files
dfstocks=dfstocks[~dfstocks.Low.str.contains("nan")]
dfstocks=dfstocks[~dfstocks.Low.str.contains("Low")]
dfstocks=dfstocks[~dfstocks.Low.str.contains("N/A")]
dfstocks=dfstocks[~dfstocks.Volume.str.contains("N/A")]
dfstocks.reset_index(drop=True, inplace=True)
dfstocks.Date.count()

229269

In [74]:
#sort dataframe based on date and Company Name
dfstocks = dfstocks.sort_values(['Date', 'Company'])
dfstocks.head(100)

,Date,Close,Volume,Open,High,Low,Company
1761,01/02/2013,78.4328,139906732,79.1171,79.2857,77.3757,AAPL
4248,01/02/2013,35.12,13767660,34.92,35.4,34.1,ABBV
6025,01/02/2013,32.05,20266410,32.3,32.45,31.64,ABT
8512,01/02/2013,69.06,4039095,67.59,69.06,67.55,ACN
10999,01/02/2013,38.34,6483720,37.92,38.73,37.92,ADBE
...,...,...,...,...,...,...,...
10747,01/02/2014,59.29,2745895,59.06,59.53,58.94,ADBE
13234,01/02/2014,168.05,1268722,167.33,170.71,166.5937,AGN
15721,01/02/2014,50.71,9196092,50.81,51.3,50.47,AIG
18208,01/02/2014,53.55,1909106,54.09,54.45,53.5,ALL


In [75]:
#Converting Datatypes
dfstocks.Date=pd.to_datetime(dfstocks.Date)
dfstocks.Close=pd.to_numeric(dfstocks.Close)
dfstocks.Volume=pd.to_numeric(dfstocks.Volume)
dfstocks.Open=pd.to_numeric(dfstocks.Open)
dfstocks.High=pd.to_numeric(dfstocks.High)
dfstocks.Low=pd.to_numeric(dfstocks.Low)
dfstocks.dtypes

Date       datetime64[ns]
Close             float64
Volume              int64
Open              float64
High              float64
Low               float64
Company            object
dtype: object

In [76]:
#dfstocks[dfstocks['Company'].str.contains("AMZN")]
sub_split=dfstocks[dfstocks['Company'].str.contains("MDLZ")]
sub_split.isnull().values.any()
sub_split.isna().values.any()
sub_split = sub_split.sort_values(['Date'])
sub_split.reset_index(drop=True, inplace=True)
sub_split.shape[0]

2487

In [77]:
sub_split

,Date,Close,Volume,Open,High,Low,Company
0,2010-02-16,19.3633,54149150,19.1695,19.3633,18.9288,MDLZ
1,2010-02-17,19.1360,24833350,19.2898,19.3499,19.0157,MDLZ
2,2010-02-18,19.2564,20606450,19.1494,19.2564,19.0324,MDLZ
3,2010-02-19,19.3299,15497250,19.2229,19.4168,19.1494,MDLZ
4,2010-02-22,19.3499,27292690,19.3098,19.4435,19.1494,MDLZ
...,...,...,...,...,...,...,...
2482,2019-12-24,54.9900,1833763,54.9400,55.1800,54.7400,MDLZ
2483,2019-12-26,54.8500,3565549,54.9700,55.0741,54.7200,MDLZ
2484,2019-12-27,55.2800,4229449,54.9300,55.3300,54.8000,MDLZ
2485,2019-12-30,55.0000,3104187,54.9200,55.0700,54.5651,MDLZ


In [0]:
# prepare training and testing data sets for LSTM based sequence modeling
#def dimitris_function(time_series, scaling=True,split_ratio=0.9):
def dimitris_function(time_series,split_ratio=0.9):

    series1 = pd.Series(time_series[time_series.columns[0]])
    series2 = pd.Series(time_series[time_series.columns[1]])
    series3 = pd.Series(time_series[time_series.columns[2]])
    series4 = pd.Series(time_series[time_series.columns[3]])
    series5 = pd.Series(time_series[time_series.columns[4]])

    values1 = series1.values
    values1 = values1.reshape((len(values1), 1))
    values2 = series2.values
    values2 = values2.reshape((len(values2), 1))
    values3 = series3.values
    values3 = values3.reshape((len(values3), 1))
    values4 = series4.values
    values4 = values4.reshape((len(values4), 1))
    values5 = series5.values
    values5 = values5.reshape((len(values5), 1))

    #scaler2 --> Close Price Scaler
    scaler1 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled1"] = scaler1.fit_transform(values1)
    scaler2 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled2"]= scaler2.fit_transform(values2)
    scaler3 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled3"]= scaler3.fit_transform(values3)
    scaler4 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled4"]= scaler4.fit_transform(values4)
    scaler5 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled5"]= scaler5.fit_transform(values5)


    time_series = time_series[["scaled1","scaled2","scaled3","scaled4","scaled5"]].values

        
    split_ratio = int(len(time_series) * split_ratio)

    train = time_series[0:split_ratio]
    test = time_series[split_ratio:len(time_series)]
    
    # return train,test,scaler1,scaler2
    return train,test,scaler2


In [0]:
#def get_seq_model(hidden_units1=40, hidden_units2=65, hidden_units3=32, drop=0.2,input_shape=(1,1)):
def get_seq_model(x_train, y_train, x_val, y_val, params):  

    # create and fit the LSTM network
    model = Sequential()

    # samples*timesteps*features
    # https://keras.io/getting-started/sequential-model-guide/
    #https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0227222

    ######!!!!!#####!!!!!
    #(train_x.shape[1],5) where 5 is the number of features is equal to :
    #(train_x.shape[1],train_x.shape[2])
    model.add(LSTM(
                   input_shape=(x_train.shape[1],x_train.shape[2]),
                   units = params['hidden_1'],  
                   #activation='relu',recurrent_activation='sigmoid',
                   return_sequences=True
    ))
    model.add(Dropout(params['dropout']))
    model.add(LSTM(
                   units = params['hidden_2'],
                   #activation='relu',recurrent_activation='sigmoid',
                   return_sequences=True
    ))
    model.add(Dropout(params['dropout']))   
    model.add(LSTM(
                   units = params['hidden_3'], 
                   #activation='relu',recurrent_activation='sigmoid',
                   return_sequences=True
    ))
    model.add(Dropout(params['dropout']))

    # readout layer. TimeDistributedDense uses the same weights for all
    # time steps.
    # model.add(TimeDistributed(Dense(1))) #number of inputs in the 1st layer
   # model.add(TimeDistributed(Dense(5)))
    model.add(TimeDistributed(Dense(5)))
    start = time.time()
    
    model.compile(loss=params['losses'], optimizer=params['optimizer'])
    
    out = model.fit(x_train, y_train,
              epochs=params['epochs'], batch_size=1,
              validation_data=[x_val, y_val],
              #epochs=200, batch_size=1,
              verbose=1)

    return out, model

In [0]:
#Parameters for the model
p = {'hidden_1':[1,4,8,12,20,40,60],
     'hidden_2':[1,4,8,12,20,40,60],
     'hidden_3':[1,4,8,12,20,40,60],
     'dropout': (0.2, 0.4, 0.05),
     'epochs': [20,80,100, 200, 300],
     'optimizer': ['Adam', "RMSprop"],
      'losses': ['logcosh', keras.losses.binary_crossentropy,"mse"]
     }

In [0]:
# print(type(sub_split.Close))
sub_split_date=sub_split[['Date']]
sub_split=sub_split[['Volume', 'Close', 'Open', 'High', 'Low']]
#sub_split.columns[1]

In [82]:
# split train and test datasets
train,test,scaler_Close = dimitris_function(sub_split,split_ratio=0.7)

#train
print(len(train))
print(train.shape[0])
print(train.shape)
print(test.shape)
print(train)

1740
1740
(1740, 5)
(747, 5)
[[0.88815584 0.02021012 0.01335138 0.01636724 0.01466068]
 [0.39046289 0.01423411 0.01651313 0.01601441 0.01695744]
 [0.31870301 0.01739958 0.01282311 0.01355248 0.01739882]
 ...
 [0.07876951 0.68608191 0.69959394 0.69877535 0.69367717]
 [0.13961111 0.67241044 0.68067072 0.68508332 0.68019791]
 [0.10580105 0.69607261 0.66910654 0.69890701 0.68019791]]


In [83]:
#We use numpy to reshape our time series into 3D tensors.
train = np.reshape(train,(1,train.shape[0],5))
test = np.reshape(test,(1,test.shape[0],5))

train_x = train[:,:-1,:]
train_y = train[:,1:,:]

test_x = test[:,:-1,:]
test_y = test[:,1:,:]

print("Data Split Complete")

print("train_x shape={}".format(train_x.shape))
print("train_y shape={}".format(train_y.shape))
print("test_x shape={}".format(test_x.shape))
print("test_y shape={}".format(test_y.shape))

Data Split Complete
train_x shape=(1, 1739, 5)
train_y shape=(1, 1739, 5)
test_x shape=(1, 746, 5)
test_y shape=(1, 746, 5)


In [0]:
# x=np.concatenate((train_x, test_x), axis=1)
# y=np.concatenate((train_y, test_y), axis=1)
# print("x shape={}".format(x.shape))
# print("y shape={}".format(y.shape))



In [0]:
#####!!!!!#####!!!!!
#(train_x.shape[1],5) where 5 is the number of features is equal to :
#(train_x.shape[1],train_x.shape[2])

In [0]:

scan_obj = talos.Scan(x=train_x,
                      y=train_y,
                      #x_val=test_x,
                      x_val=train_x,
                      #y_val=test_y,
                      y_val=train_y,
                      params=p,
                      #val_split=0.3,
                      model=get_seq_model,
                      experiment_name='gamw_ton_alexiou')
                      #fraction_limit=0.1)




  0%|          | 0/10290 [00:00<?, ?it/s]

Train on 1 samples, validate on 1 samples
Epoch 1/20
1/1 [==============================] - 5s 5s/sample - loss: 0.0655 - val_loss: 0.0652
Epoch 2/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0652 - val_loss: 0.0649
Epoch 3/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0649 - val_loss: 0.0646
Epoch 4/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0646 - val_loss: 0.0644
Epoch 5/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0644 - val_loss: 0.0641
Epoch 6/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0641 - val_loss: 0.0638
Epoch 7/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0638 - val_loss: 0.0635
Epoch 8/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0635 - val_loss: 0.0632
Epoch 9/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0633 - val_loss: 0.0630
Epoch 10/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0630 - val_


  0%|          | 1/10290 [01:08<195:27:33, 68.39s/it]

Train on 1 samples, validate on 1 samples
Epoch 1/20
1/1 [==============================] - 5s 5s/sample - loss: 0.0655 - val_loss: 0.0643
Epoch 2/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0643 - val_loss: 0.0634
Epoch 3/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0634 - val_loss: 0.0626
Epoch 4/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0626 - val_loss: 0.0619
Epoch 5/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0619 - val_loss: 0.0613
Epoch 6/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0613 - val_loss: 0.0606
Epoch 7/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0607 - val_loss: 0.0600
Epoch 8/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0600 - val_loss: 0.0594
Epoch 9/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0594 - val_loss: 0.0588
Epoch 10/20
1/1 [==============================] - 3s 3s/sample - loss: 0.0589 - val_


  0%|          | 2/10290 [02:16<195:21:18, 68.36s/it]

...

Train on 1 samples, validate on 1 samples
Epoch 1/20
1/1 [==============================] - 5s 5s/sample - loss: 2.5912 - val_loss: 2.1944
Epoch 2/20
1/1 [==============================] - 4s 4s/sample - loss: 2.2420 - val_loss: 2.0279
Epoch 3/20
1/1 [==============================] - 4s 4s/sample - loss: 2.0813 - val_loss: 1.9059
Epoch 4/20
1/1 [==============================] - 4s 4s/sample - loss: 1.9503 - val_loss: 1.8008
Epoch 5/20
1/1 [==============================] - 4s 4s/sample - loss: 1.7780 - val_loss: 1.6843
Epoch 6/20
1/1 [==============================] - 4s 4s/sample - loss: 1.5557 - val_loss: 0.9871
Epoch 7/20
1/1 [==============================] - 4s 4s/sample - loss: 1.2200 - val_loss: 0.9041
Epoch 8/20
1/1 [==============================] - 3s 3s/sample - loss: 1.0345 - val_loss: 0.8584
Epoch 9/20
1/1 [==============================] - 4s 4s/sample - loss: 0.9372 - val_loss: 0.8252
Epoch 10/20
1/1 [==============================] - 4s 4s/sample - loss: 0.8791 - val_


  6%|▌         | 573/10290 [10:38:33<204:04:34, 75.61s/it]

Train on 1 samples, validate on 1 samples
Epoch 1/20
1/1 [==============================] - 5s 5s/sample - loss: 2.4574 - val_loss: 1.6611
Epoch 2/20
1/1 [==============================] - 3s 3s/sample - loss: 1.6105 - val_loss: 1.5085
Epoch 3/20
1/1 [==============================] - 4s 4s/sample - loss: 1.3292 - val_loss: 0.7530
Epoch 4/20
1/1 [==============================] - 3s 3s/sample - loss: 0.8704 - val_loss: 0.7373
Epoch 5/20
1/1 [==============================] - 4s 4s/sample - loss: 0.8347 - val_loss: 0.7236
Epoch 6/20
1/1 [==============================] - 3s 3s/sample - loss: 0.8058 - val_loss: 0.7053
Epoch 7/20
1/1 [==============================] - 3s 3s/sample - loss: 0.7766 - val_loss: 0.6946
Epoch 8/20
1/1 [==============================] - 3s 3s/sample - loss: 0.7610 - val_loss: 0.6849
Epoch 9/20
1/1 [==============================] - 3s 3s/sample - loss: 0.7558 - val_loss: 0.6744
Epoch 10/20
1/1 [==============================] - 3s 3s/sample - loss: 0.7283 - val_


  6%|▌         | 574/10290 [10:39:48<203:09:11, 75.27s/it]

Train on 1 samples, validate on 1 samples
Epoch 1/20
1/1 [==============================] - 5s 5s/sample - loss: 0.1507 - val_loss: 0.1400
Epoch 2/20
1/1 [==============================] - 4s 4s/sample - loss: 0.1401 - val_loss: 0.1310
Epoch 3/20
1/1 [==============================] - 4s 4s/sample - loss: 0.1312 - val_loss: 0.1229
Epoch 4/20
1/1 [==============================] - 4s 4s/sample - loss: 0.1232 - val_loss: 0.1152
Epoch 5/20
1/1 [==============================] - 4s 4s/sample - loss: 0.1155 - val_loss: 0.1076
Epoch 6/20
1/1 [==============================] - 4s 4s/sample - loss: 0.1079 - val_loss: 0.1000
Epoch 7/20
1/1 [==============================] - 4s 4s/sample - loss: 0.1000 - val_loss: 0.0921
Epoch 8/20
1/1 [==============================] - 4s 4s/sample - loss: 0.0923 - val_loss: 0.0837
Epoch 9/20
1/1 [==============================] - 5s 5s/sample - loss: 0.0845 - val_loss: 0.0748
Epoch 10/20
1/1 [==============================] - 4s 4s/sample - loss: 0.0765 - val_

In [0]:
analyze_object = talos.Analyze(scan_obj)


In [0]:
scan_obj.data.head

In [0]:
r = talos.Reporting(scan_obj)

In [0]:
p = talos.Predict(scan_obj)

In [0]:
r.data.sort_values('val_loss')

In [0]:
train_y

In [0]:
# train the model
#https://medium.com/@rajath95/how-to-save-and-upload-deep-learning-machine-learning-models-in-google-colab-using-google-drive-eb2901937ae
# seq_lstm_model.fit(train_x, train_y,
#                 epochs=2, batch_size=1,
#                 #epochs=200, batch_size=1,
#                 verbose=1)
# print("Model Fit Complete")

In [0]:
# train fit performance
trainPredict = seq_lstm_model.predict(train_x)
trainScore = math.sqrt(mean_squared_error(train_y[0], trainPredict[0]))
print('Train Score: %.2f RMSE' % (trainScore))

In [0]:
trainPredict

In [0]:
# Pad input sequence
testPredict = pad_sequences(test_x,maxlen=train_x.shape[1],padding='post',dtype='float64')

In [0]:
# forecast values
testPredict = seq_lstm_model.predict(testPredict)
testPredict

In [0]:
#Drop all other variables except Close Price Output
testPredict=np.delete(testPredict, [2,3,4], 2)
trainPredict=np.delete(trainPredict, [2,3,4], 2)
# testPredict=np.delete(testPredict, 3, 2)
# trainPredict=np.delete(trainPredict, 3, 2)
# testPredict=np.delete(testPredict, 4, 2)
# trainPredict=np.delete(trainPredict, 4, 2)
testPredict=np.delete(testPredict, 0, 2)
trainPredict=np.delete(trainPredict, 0, 2)

In [0]:
testPredict

In [0]:
trainPredict


In [0]:
type(testPredict)

In [0]:
# # inverse transformation of the Close Price
trainPredict = scaler_Close.inverse_transform(trainPredict.\
                                        reshape(-1,1))
testPredict = scaler_Close.inverse_transform(testPredict.\
                                       reshape(-1,1))

In [0]:
trainPredict

In [0]:
# plot the true and forecasted values
split_ratio = len(trainPredict)+1

plt.plot(sub_split_date.Date,
          sub_split.Close.values,c='black',
          alpha=0.3,label='True Data')

plt.plot(sub_split_date.Date[1:split_ratio],
          trainPredict,label='Training Fit',c='g')

plt.plot(sub_split_date.Date[split_ratio+1:],
          testPredict[:test_x.shape[1]],label='Testing Forecast')
plt.title('Forecast Plot')
plt.legend()
#plt.show()
plt.rcParams["figure.figsize"] = (20,10)
plt.show()